<a href="https://colab.research.google.com/github/Bryan-Az/Advanced-Prompt-Engineering/blob/main/notebooks/prompt_techniques/RASCEF_REACT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Applying RASCEF and REACT to Prompt Template Generation

## Imports and Installs

In [1]:
%%capture
!pip install datasets

In [2]:
import json
import datasets
import openai
from google.colab import userdata
import os

## Loading the Sample Templates

In [6]:
%%capture
# downloading the react sample json from github
! wget https://raw.githubusercontent.com/Bryan-Az/Advanced-Prompt-Engineering/refs/heads/main/prompts/ReACT_sample_structure.json -O ReACT_sample_structure.json
! wget https://raw.githubusercontent.com/Bryan-Az/Advanced-Prompt-Engineering/refs/heads/main/prompts/RASCEF_sample_structure.json -O RASCEF_sample_structure.json

In [7]:
#loading the sample templates as json
with open('ReACT_sample_structure.json', 'r') as file:
    ReACT_sample_structure = json.load(file)
with open('RASCEF_sample_structure.json', 'r') as file:
    RASCEF_sample_structure = json.load(file)

In [5]:
display(ReACT_sample_structure)

{'prompt': '[Initial question or task]',
 'actions': [{'name': '[Action Name]',
   'description': '[Brief description of what the action does]',
   'result_format': '[Format of the result]'}],
 'action_requests': [{'condition': '[Condition for triggering the action]',
   'requested_action': '[Name of the action to perform]'}]}

In [9]:
display(RASCEF_sample_structure)

{'role': 'Expert',
 'action': 'Analyze and [Desired Outcome]',
 'steps': ['1. [Step 1 Description]',
  '2. [Step 2 Description]',
  '3. [Step 3 Description]'],
 'context': '[Brief description of the situation or data]',
 'examples': [{'input_data': '[Example input format]',
   'expected_output': '[Desired output format]'}],
 'format': '[How the output should be structured]'}

## Initializing the OpenAI Model API Class

In [10]:
openai_key = userdata.get("openai")
os.environ["OPENAI_API_KEY"] = openai_key

In [11]:
model_name = 'gpt-3.5-turbo'
def get_completion(prompt, model=model_name):
  """Generates text using the OpenAI API.

  Args:
    prompt: The prompt to send to the API.
    model: The name of the model to use.

  Returns:
    The generated text.
  """
  response = openai.chat.completions.create(
      model=model,
      messages=[{"role": "user", "content": prompt}],
      temperature=0.01
  )
  return response.choices[0].message.content

## Using the Model API to Generate ICL Prompts for 3 Domains
As I already generated prompt templates in the fields of healthcare, software, math, finance, and data analysis - I will ask the model to generate 8 more prompt templates in distinct fields for both RASCEF and ReACT strategies.

In [56]:
# calling the OpenAI API to use this template to generate RASCEF prompts
RASCEF_prompt_generation_prompt = f"Can you help me use this sample json template to generate 8 RASCEF style JSON prompts in the fields of construction, sculpting, gardening, farming, environmental science, and law enforcement?. Remember to change the key name of the domain to reflect the field. Template: {RASCEF_sample_structure}"

rascef_response = get_completion(RASCEF_prompt_generation_prompt)

In [15]:
# calling the OpenAI API to use this template to generate ReACT prompts
react_prompt_generation_prompt= f"Can you help me use this sample json template to generate 8 ReACT style JSON prompts for Management, Fashion, Food/Beverage, Healthcare, Journalism, Library Science, Music Therapy, and Social Work? Remember to change the key name of the domain to reflect the field. Template: {ReACT_sample_structure}"
react_response = get_completion(react_prompt_generation_prompt)

In [66]:
# using regex to select the json from the text output
import re
def extract_json_from_text(text):
    json_objects = []
    while True:  # Iterate until no more JSON objects are found
        # find many instances of inner json objects
        matches = re.search(r'({.*?})', text, re.DOTALL)
        if matches is None:
            break  # No more JSON objects, exit loop
        json_str = matches.group(1)  # Extract matched JSON string
        text = text.replace(json_str, '', 1)  # Remove matched JSON from text
        try:
            json_obj = json.loads(json_str)
            json_objects.append(json_obj)
        except:
            pass  # Ignore invalid JSON strings
        json_objects.append(text)

    return json_objects
# applying it on the 2 prompt styles
prompts = [react_response, rascef_response]
extracted_rascef_prompts = extract_json_from_text(prompts[1])
extracted_react_prompts = extract_json_from_text(prompts[0])

In [72]:
print(len(extracted_rascef_prompts))
print(len(extracted_react_prompts))

6
16


In [76]:
# writing the output for both rascef and react prompts by writing to a new json file at prompts/generated_prompts
with open('prompts/generated_prompts/6_rascef_prompts.json', 'w') as file:
    json.dump(extracted_rascef_prompts, file, indent=4)
with open('prompts/generated_prompts/16_react_prompts.json', 'w') as file:
    json.dump(extracted_react_prompts, file, indent=4)